In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

### Gell-Mann Matrices

$$\displaystyle \hat{t}^{(1)} = \left(\begin{array}{ccc} 0 & 1 & 0 \\ 1 & 0 & 0\\ 0 & 0 & 0\end{array}\right) \;\;\;\;\;\;\;\;\;\; 
\hat{t}^{(2)} = \left(\begin{array}{ccc} 0 & -i & 0 \\ i & 0 & 0\\ 0 & 0 & 0\end{array}\right)\;\;\;\;\;\;\;\;\;\;\hat{t}^{(3)} = \left(\begin{array}{ccc} 1 & 0 & 0 \\ 0 & -1 & 0\\ 0 & 0 & 0\end{array}\right)$$
$$\hat{t}^{(4)} = \left(\begin{array}{ccc} 0 & 0 & 1 \\ 0 & 0 & 0\\ 1 & 0 & 0\end{array}\right)\;\;\;\;\;\;\;\;\;\;
\displaystyle \hat{t}^{(5)} = \left(\begin{array}{ccc} 0 & 0 & -i \\ 0 & 0 & 0\\ i & 0 & 0\end{array}\right) \;\;\;\;\;\;\;\;\;\;
\hat{t}^{(6)} = \left(\begin{array}{ccc} 0 & 0 & 0 \\ 0 & 0 & 1\\ 0 & 1 & 0\end{array}\right)$$
$$\hat{t}^{(7)} = \left(\begin{array}{ccc} 0 & 0 & 0 \\ 0 & 0 & -i\\ 0 & i & 0\end{array}\right)\;\;\;\;\;\;\;\;\;\; \hat{t}^{(8)} = \frac{1}{\sqrt{3}}\left(\begin{array}{ccc} 1 & 0 & 0 \\ 0 & 1 & 0\\ 0 & 0 & -2\end{array}\right)\\$$

For convenience, these are packaged as slices of a three-dimensional array. The $3\times 3$ identity matrix is also defined here and denoted $\hat{t}^{(0)}$.

In [ ]:
gm_mats = np.zeros((9, 3, 3), dtype=np.complex128)
gm_mats[0] = np.eye(3)
gm_mats[1] = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])
gm_mats[2] = np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]])
gm_mats[3] = np.diag([1, -1, 0])
gm_mats[4] = np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]])
gm_mats[5] = np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]])
gm_mats[6] = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
gm_mats[7] = np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]])
gm_mats[8] = np.diag([1, 1, -2]) / np.sqrt(3)

### Color Basis States
The three color basis states are red, green, and blue. For our purposes, they will be defined as such:
$$
\displaystyle |r\rangle = \left(\begin{array}{ccc} 1 \\ 0 \\ 0 \end{array}\right) \;\;\;\;\;\; |b\rangle = \left(\begin{array}{ccc} 0 \\ 1 \\ 0 \end{array} \right) \;\;\;\;\;\; |g\rangle = \left(\begin{array}{ccc} 0 \\ 0 \\ 1 \end{array} \right) \;\;\;\;\;\;

$$

### Two-Body Interactions

The Hamiltonian for interactions between two sources of color charge is:
$$J \displaystyle \sum_{\alpha=1}^8 \hat{t}_1^{(\alpha)} \otimes \hat{t}_2^{(\alpha)}$$
where $\hat{t}^{(i)}$ is the $i$-th Gell-Mann matrix, and $J$ is some positive, real constant.



In [ ]:
J = 1.0 # two-body interaction strength

H = np.zeros((9, 9), dtype=np.complex128, requires_grad=False)

for i in range(1, len(gm_mats)):
    H += J * np.kron(gm_mats[i], gm_mats[i])

In [ ]:
w, v = np.linalg.eigh(H)

w[0]

In [ ]:
v[:, 0]

# VQE for two color sources
There are two ways going about VQE for our Hamiltonian. The first solution is to create a uniform superposition of all basis states and use a general ansatz to tune the amplitudes of each basis state (i.e. general ansatz).

The other solution is to create an ansatz that creates a superposition of all basis states that are present in the lowest-energy eigenstate of the Hamiltonian, and then tune the amplitude (i.e. problem specific ansatz).

#### General Ansatz

In [ ]:
# For the uniform superposition, we need the qutrit Hadamard gate. However, this is
# not implemented yet, so I will use `qml.QutritUnitary`
omega = np.exp(2 * np.pi * 1j / 3)
Hadamard = np.array(
    [
        [1, 1, 1],
        [1, omega, omega**2],
        [1, omega**2, omega]
    ],
    requires_grad=False
) * 1j / np.sqrt(3)


def create_uniform_superposition():
    qml.QutritUnitary(Hadamard, wires=0)
    qml.QutritUnitary(Hadamard, wires=1)

def rot(weights, wires):
    qml.TRY(weights[0], wires=wires, subspace=[0, 1])
    qml.TRY(weights[1], wires=wires, subspace=[0, 2])
    qml.TRY(weights[2], wires=wires, subspace=[0, 1])
    qml.TRZ(weights[3], wires=wires, subspace=[0, 2])
    qml.TRZ(weights[4], wires=wires, subspace=[0, 1])
    qml.TRY(weights[5], wires=wires, subspace=[0, 1])
    qml.TRY(weights[6], wires=wires, subspace=[0, 2])
    qml.TRY(weights[7], wires=wires, subspace=[0, 1])


def ansatz(params, check_state=False):
    # create_uniform_superposition()

    # Trainable layer
    rot(params[0], 0)
    rot(params[1], 1)
    qml.TAdd(wires=[1, 0])
    
    rot(params[2], 0)
    rot(params[3], 1)
    qml.TAdd(wires=[0, 1])

    if check_state:
        return qml.state()

    return qml.expval(qml.THermitian(H, wires=[0, 1]))

In [ ]:
dev = qml.device("default.qutrit", wires=2)
ansatz_qnode = qml.QNode(ansatz, dev, diff_method="parameter-shift")

weights = np.random.normal(size=(4, 8))

opt = qml.GradientDescentOptimizer(stepsize=0.05)

n_its = 100

losses = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(ansatz_qnode, weights)
    losses.append(_loss)
    
    if it % 5 == 0:
        print(f"Loss at iteration {it:2} = {_loss: .8f}")

In [ ]:
plt.plot(losses)

In [ ]:
losses[-1]

In [ ]:
w, v = np.linalg.eigh(H)

w[0]

In [ ]:
v[:, 0]

In [ ]:
ansatz_qnode(weights, check_state=True)

#### Problem Specific Ansatz

Earlier, we saw that the lowest-energy eigenstate of the Hamiltonian is
$$\displaystyle \frac{1}{\sqrt{2}} \begin{bmatrix}
0\\
0\\
-1\\
0\\
0\\
0\\
1\\
0\\
0\\
\end{bmatrix} = \frac{1}{\sqrt{2}} \left( -|rg\rangle + |gr\rangle\right)$$

Let's create an ansatz that is more optimal for creating this specific state

In [ ]:
def ansatz_specific(params, check_state=False):
    qml.TRY(params[0], wires=1, subspace=[0, 2])
    qml.TAdd(wires=[1, 0])
    qml.TRY(params[1], wires=0, subspace=[0, 2])

    if check_state:
        return qml.state()

    return qml.expval(qml.THermitian(H, wires=[0, 1]))

In [ ]:
ansatz_specific_qnode = qml.QNode(ansatz_specific, dev, diff_method="parameter-shift")

weights = np.random.normal(size=4)

opt = qml.GradientDescentOptimizer(stepsize=0.05)

n_its = 100

losses = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(ansatz_specific_qnode, weights)
    losses.append(_loss)
    
    if it % 5 == 0:
        print(f"Loss at iteration {it:2} = {_loss: .8f}")

In [ ]:
plt.plot(losses)

In [ ]:
losses[-1]

In [ ]:
w, v = np.linalg.eigh(H)

w[0]

In [ ]:
v[:, 0]

In [ ]:
ansatz_specific_qnode(weights, check_state=True)

# Hamiltonian simulation for two color sources